<a href="https://colab.research.google.com/github/musstafa08-bug/ChatBot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tflearn

In [2]:
import numpy as np
import random
import json
import tensorflow as tf
import tflearn
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with open('/content/intents.json') as json_data:
  intents = json.load(json_data)

In [4]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [5]:
words = []
classes= []
document = []
ignore = ['?', ',', '-', '_', '@', '!', '#', '/', '.' ]

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    document.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(document), "Documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

31 Documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [7]:
train = []
output = []
output_empty = [0] * len(classes)
for doc in document:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # op is '1' for current tag and '0' for rest of the other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    train.append([bag, output_row])

random.shuffle(train)
train = np.array(train)

# training list
train_x = list(train[:, 0])
train_y = list(train[:, 1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
print(len(train_x))
print(len(train_y[0]))

1767
9


In [9]:
#tf.reset_default_graph()
nn = tflearn.input_data(shape = [None, len(train_x[0])])
nn = tflearn.fully_connected(nn, 10)
nn = tflearn.fully_connected(nn, 10)
nn = tflearn.fully_connected(nn, len(train_y[0]), activation = 'softmax')
nn = tflearn.regression(nn)

model = tflearn.DNN(nn, tensorboard_dir = 'tflearn_logs')
model.fit(train_x, train_y, n_epoch = 800, batch_size = 8, show_metric = True)
model.save('model.tflearn')

Training Step: 45353  | total loss: 0.00000 | time: 0.318s
| Adam | epoch: 206 | loss: 0.00000 - acc: 1.0000 -- iter: 0384/1767


In [12]:
import pickle
pickle.dump({'words': words, 'classes':classes, 'train_x': train_x, 'train_y': train_y}, open("trained_data", "wb"))

In [13]:
data = pickle.load(open("/content/trained_data", "rb"))
words = data["words"]
classes = data["classes"]
train_x = data["train_x"]
train_y = data["train_y"]

In [14]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [30]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [31]:
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence, words, show_details = False):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] =1
        if show_details:
          print("found in bag: %s" %w)
  return(np.array(bag))


In [36]:
thres = 0.30
def classify(sentence):
  results = model.predict([bag_of_words(sentence, words)])[0]
  results = [[i,r] for i, r in enumerate(results) if r>thres]
  results.sort(key = lambda x: x[1], reverse = True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))

            results.pop(0)

In [37]:
classify('What are your hours of labour')

[('hours', 1.0)]

In [38]:
response('What are your hours of labour')

We're open every day 9am-9pm


In [45]:
response('Do you accept Credit Card?')


We accept VISA, Mastercard and AMEX


In [39]:
classify('food')

[('deliveryoption', 0.82857347)]

In [40]:
response('food')

We have home delivery options through UBER Eats and Zomato


In [42]:
response('What is menu for today?')


Our speciality for today is Chicken Tikka


In [44]:
response("Hi there!")


Hello, thanks for visiting


In [46]:
response('Thanks')

Happy to help!
